### GWO Optimized XGBoost 

#### Importing Libraries

In [ ]:
import numpy as np 
from numpy import array
import pandas as pd 
import matplotlib.pyplot as plt  
from matplotlib import pyplot   
import seaborn as sns  
import random
import os
import time

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler,LabelEncoder  
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
from keras.models import Model
from keras import regularizers

from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, classification_report  
from sklearn.metrics import auc, f1_score, roc_curve, roc_auc_score  
from sklearn.metrics import precision_score, recall_score, accuracy_score
%matplotlib inline

from sklearn.metrics import accuracy_score
import xgboost as xgb
import matplotlib.patches as patches
import tensorflow as tf
import keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from keras.layers import Input, Dense, BatchNormalization
from sklearn.metrics import mean_absolute_error, mean_squared_error


### Dataset Import

In [ ]:
data_path = 'C:/Desktop/full-cicids2017/MachineLearningCVE/'  

ls = []

for filename in os.listdir(data_path):
    if filename.endswith('.csv'):
        print(filename)
        
        file_path = os.path.join(data_path, filename)
        
        df_ids2017 = pd.read_csv(file_path)
        
        ls.append(df_ids2017)
        
        print(f'Shape: {df_ids2017.shape}. Attack Type: {df_ids2017[' Label'].unique()}")

### Pre-processing and other methods

In [ ]:
df_ids2017 = pd.concat(ls)
df_ids2017.head(3)
assert df_ids2017.shape[1] == 79 

In [ ]:
df_ids2017.columns = df_ids2017.columns.str.strip()
print("original length:", len(df_ids2017))

In [ ]:
df_ids2017.isnull().sum().sum()

In [ ]:
# removing missing values
df_ids2017.replace([np.inf, -np.inf], np.nan, inplace=True)
df_ids2017.dropna(inplace=True)
print("after droping null values, the length of df:", len(df_ids2017))

In [ ]:
df_ids2017.Label.value_counts()

In [ ]:
df_ids2017['Label'].unique()

In [ ]:
df_ids2017.Label.replace("Web.*", "Web Attack", regex=True, inplace=True)
df_ids2017.Label.replace(r'.*Patator$', "Brute Force", regex=True,inplace=True)
df_ids2017.Label.replace(["DoS GoldenEye", "DoS Hulk", "DoS Slowhttptest", "DoS slowloris"], "DoS", inplace=True)
df_ids2017.Label.value_counts()

In [ ]:
df_ids2017['Label'].unique()

In [ ]:
df_ids2017.head(2)

In [ ]:
df_ids2017.Label.replace("Web.*", "Web Attack", regex=True, inplace=True)
df_ids2017.Label.replace(r'.*Patator$', "Brute Force", regex=True,inplace=True)
df_ids2017.Label.replace(["DoS GoldenEye", "DoS Hulk", "DoS Slowhttptest", "DoS slowloris"], "DoS", inplace=True)
df_ids2017.Label.value_counts()

In [ ]:
df_ids2017['Label'].unique()

In [ ]:
df_ids2017['Label'].replace({'PortScan': 'attack', 'Web Attack':'attack',
                            'Brute Force': 'attack',
                            'DDoS':'attack' , 'Bot':'attack',
                            'Infiltration':'attack',
                            'DoS':'attack', 'Heartbleed':'attack'}, inplace=True)

In [ ]:
df_ids2017['Label'].unique()

In [ ]:
pd.value_counts(df_ids2017['Label'].values)

In [ ]:
le_target = LabelEncoder()
df_ids2017.Label =le_target.fit_transform(df_ids2017.Label)

In [ ]:
pd.value_counts(df_ids2017['Label'].values)

In [ ]:
X = df_ids2017.drop('Label', axis=1)
y = df_ids2017['Label']

In [ ]:
# Data Balancing  
rus = RandomUnderSampler(sampling_strategy={0: 1414698}, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

smote = SMOTE(sampling_strategy={1: 1414698}, random_state=42)
x_train_input, y_train_target = smote.fit_resample(X_resampled, y_resampled)

# distribution of classes after balancing
print(pd.Series(y_train_target).value_counts())

In [ ]:
# data split
x_train_input, x_test_input, y_train_target, y_test_target = train_test_split(x_train_input,  y_train_target,test_size=0.30,random_state=42, stratify= y_train_target)
print("after spliting the data:\n")
print("training data input length:", len(x_train_input))
print("test data input  length:", len(x_test_input))

In [ ]:
x_train_input.shape, y_train_target.shape, x_test_input.shape, y_test_target.shape

In [ ]:
def correlation(dataset, threshold):
    col_cor = set()  
    corr_matrix = x_train_input.corr().abs()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: 
                colname = corr_matrix.columns[i]  
                col_cor.add(colname)
    return col_cor

In [ ]:
corr_features = correlation(x_train_input, 0.96) 
len(set(corr_features))

In [ ]:
x_train_input.drop(corr_features,axis=1, inplace=True)
x_test_input.drop(corr_features,axis=1,inplace=True)

In [ ]:
y_train_target.value_counts() 

In [ ]:
y_test_target.value_counts() 

In [ ]:
x_train_input_0, y_train_target_1 = y_train_target.value_counts()[0] / len(y_train_target) , y_train_target.value_counts()[1] / len(y_train_target)
x_test_input_0, y_test_target_1 = y_test_target.value_counts()[0] / len(y_test_target), y_test_target.value_counts()[1] /len(y_test_target) 

print("In Train: there are {} % of class 0 and {} % of class 1".format(x_train_input_0, y_train_target_1))
print("In Test: there are {} % of class 0 and {} % of class 1".format(x_test_input_0, y_test_target_1))

In [ ]:
# Plotting target label distribution
plt.figure()
plt.title("class distribution of train and test dataset")
y_train_target.value_counts().plot(kind="bar", color='c', label="y_train_target")
y_test_target.value_counts().plot(kind="bar", color='blue', label="y_test_targe")
plt.xlabel("Class")
plt.ylabel("Count")
plt.legend()
plt.show()

In [ ]:
# checking for null values after pre-processing
x_train_input.isnull().sum().sum()

In [ ]:
x_train_input.isnull().sum().sum(), x_test_input.isnull().sum().sum()

In [ ]:
x_train_input.dtypes.value_counts()
x_train_input.describe().round()

In [ ]:
# normalization
scaler_t = MinMaxScaler()
x_train_input_SC = scaler_t.fit_transform(x_train_input)
x_test_input_SC = scaler_t.transform(x_test_input)

In [ ]:
x_train_input_SC.max().round()
x_train_input_SC.min().round()

### Autoencoder Model

In [ ]:
# loss function definition
def custom_loss(inputs, outputs, encoder_weights, decoder_weights, delta=1.0, alpha=0.001, beta=0.001, gamma=0.005, epsilon=1e-10):
    term1 = tf.reduce_sum((delta * tf.square(inputs - outputs)) / (delta + tf.square(inputs - outputs)))
    term2 = tf.reduce_sum(tf.square(inputs - outputs) / (tf.square(inputs - outputs) + delta))
    els_loss = term1 + term2 

    encoder_layer_weights = encoder_weights[0]
    decoder_layer_weights = decoder_weights[0]

    sparse_reg_term = alpha * tf.reduce_sum(tf.abs(encoder_layer_weights)) + alpha * tf.reduce_sum(tf.abs(decoder_layer_weights))

    l2_reg_term = 0.5 * beta * tf.reduce_sum([tf.nn.l2_loss(w) for w in encoder_layer_weights]) + 0.5 * beta * tf.reduce_sum([tf.nn.l2_loss(w) for w in decoder_layer_weights])

    # Total loss
    total_loss = els_loss + sparse_reg_term + l2_reg_term
    
    return total_loss

def custom_loss_wrapper(encoder_weights, decoder_weights, delta=1.0, alpha=0.001, beta=0.001):
    def loss(inputs, outputs):
        return custom_loss(inputs, outputs, encoder_weights, decoder_weights, delta, alpha, beta)
    return loss

def create_autoencoder(input_dim, encoding_dim, l1_reg, l2_reg, learning_rate):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(64, activation='relu', activity_regularizer=regularizers.l1_l2(l1=l1_reg, l2=l2_reg))(input_layer)
    encoded = BatchNormalization()(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Dense(encoding_dim, activation='relu')(encoded)
    
    decoded = Dense(32, activation='relu')(encoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dense(64, activation='relu')(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    encoder = Model(inputs=input_layer, outputs=encoded)

    encoder_weights = [layer.trainable_weights for layer in autoencoder.layers if 'dense' in layer.name and layer.output_shape == (None, encoding_dim)]
    decoder_weights = [layer.trainable_weights for layer in autoencoder.layers if 'dense' in layer.name and layer.output_shape == (None, input_dim)]

    optimizer = Adam(learning_rate=learning_rate)
    
    autoencoder.compile(optimizer=optimizer, loss=custom_loss_wrapper(encoder_weights, decoder_weights))
    
    return autoencoder, encoder

def random_search(X_train_input_SC, hyperparameter_ranges, n_iter):
    best_params = None
    best_mse = float('inf')

    for _ in range(n_iter):
        config = {param: np.random.choice(options) for param, options in hyperparameter_ranges.items()}
        model, _ = create_autoencoder(input_dim=X_train_input_SC.shape[1], encoding_dim=config['encoding_dim'],
                                   l1_reg=config['l1_reg'], l2_reg=config['l2_reg'], learning_rate=config['learning_rate'])
        history = model.fit(X_train_input_SC, X_train_input_SC, epochs=config['num_epochs'],
                            batch_size=config['batch_size'], verbose=0, validation_split=0.3)
        mse = history.history['val_loss'][-1]

        if mse < best_mse:
            best_mse = mse
            best_params = config

    return best_params, best_mse

# Hyperparameter search space
hyperparameter_ranges = {
    'batch_size': [32, 64, 128, 256, 1024],  
    'l1_reg': [0.1, 0.01, 0.001, 0.0001],
    'l2_reg': [0.1, 0.01, 0.001, 0.0001], 
    'encoding_dim': [16], 
    'num_epochs': [50, 75, 100, 150, 200],
    'learning_rate': [0.0001, 0.001, 0.01, 0.02] 
}

best_params, best_mse = random_search(x_train_input_SC, hyperparameter_ranges, n_iter=5)

print("Best Configuration:", best_params)
print("Best MSE:", best_mse)

best_autoencoder, best_encoder = create_autoencoder(input_dim=x_train_input_SC.shape[1], encoding_dim=best_params['encoding_dim'],
                                                    l1_reg=best_params['l1_reg'], l2_reg=best_params['l2_reg'], learning_rate=best_params['learning_rate'])

history = best_autoencoder.fit(x_train_input_SC, x_train_input_SC, epochs=best_params['num_epochs'],
                                batch_size=best_params['batch_size'], verbose=0, shuffle=True, validation_split=0.3)

encoded_features = best_encoder.predict(x_test_input_SC)
print("Encoded Features Shape:", encoded_features.shape)

X_Mae = best_autoencoder.predict(x_test_input_SC)
mae_test = np.mean(np.abs(x_test_input_SC - X_Mae))
print("Test MAE:", mae_test)

mse_test = best_autoencoder.evaluate(x_test_input_SC, x_test_input_SC, verbose=0)
print("Test MSE:", mse_test)

In [ ]:
encoded_features_train_EDA = best_autoencoder.predict(x_train_input_SC)
encoded_features_test_EDA = best_autoencoder.predict(x_test_input_SC)

In [ ]:
scatter = plt.scatter(encoded_features_train_EDA[:, 1], encoded_features_train_EDA[:, 0], c=y_train_target, cmap='viridis')
plt.colorbar(scatter, label='Class')
plt.title('Encoded Features Visualization')
plt.xlabel('Encoded Feature 1')
plt.ylabel('Encoded Feature 2')
#plt.savefig('FeE_Vclass2_01.png')
plt.show()

In [ ]:
reconstruction_error = mean_absolute_error(x_train_input_SC, encoded_features_train_EDA, multioutput='raw_values')
plt.figure(figsize=(10, 6))
plt.hist(reconstruction_error, bins=50, density=True, alpha=0.7, color='blue')
plt.title('Reconstruction Error Distribution')
plt.xlabel('Reconstruction Error (MAE)')
plt.ylabel('Frequency')
#plt.savefig('MAE_Encode_Recon_Error.png')
plt.show()

In [ ]:
reconstruction_error = mean_squared_error(x_train_input_SC, encoded_features_train_EDA, multioutput='raw_values')
plt.figure(figsize=(10, 6))
plt.hist(reconstruction_error, bins=50, density=True, alpha=0.7, color='blue')
plt.title('Reconstruction Error Distribution')
plt.xlabel('Reconstruction Error (MSE)')
plt.ylabel('Frequency')
#plt.savefig('MSE_Encode_REcon_Error.png')
plt.show()

In [ ]:
encoded_features_train_ENC = best_encoder.predict(x_train_input_SC)
encoded_features_test_ENC = best_encoder.predict(x_test_input_SC)

In [ ]:
scatter = plt.scatter(encoded_features_train_ENC[:, 1], encoded_features_train_ENC[:, 0], c=y_train_target, cmap='viridis')
plt.colorbar(scatter, label='Class')
plt.title('Encoded Features Visualization')
plt.xlabel('Encoded Feature 1')
plt.ylabel('Encoded Feature 2')
#plt.savefig('CICIDS2017_COMP_16_FeE_Vclass2.png')

### XGBoost hyperparameter Optimization(GWO) 

In [ ]:
def objective_function(hyperparameters):
    xgb_model = xgb.XGBClassifier(**hyperparameters)
    xgb_model.fit(encoded_features_train_ENC, y_train_target)
    predictions = xgb_model.predict(encoded_features_test_ENC)
    accuracy = accuracy_score(y_test_target, predictions)
    return accuracy

def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        hyperparameters = {
            'learning_rate': np.random.uniform(0.01, 0.3),
            'max_depth': np.random.randint(4, 20),
            'subsample': np.random.uniform(0.6, 1.0),
            'min_child_weight': np.random.uniform(0.5, 2.0),
            'n_estimators': np.random.randint(100, 300),
            'max_leaf_nodes': np.random.randint(1, 9),
            'colsample_bytree': np.random.uniform(0.5, 0.9),
            'gamma': 0.1,
            'scale_pos_weight':np.random.uniform(1, 20),
            'seed': 42
        }
        population.append(hyperparameters)
    return population

def update_wolf_positions(population, alpha, beta, delta, a, A, C):
    for wolf in population:
        for key in wolf:
            if key in alpha:
                D_alpha = abs(C * alpha[key] - wolf[key])
                X1 = alpha[key] - A * D_alpha
                wolf[key] = X1
            elif key in beta:
                D_beta = abs(C * beta[key] - wolf[key])
                X2 = beta[key] - A * D_beta
                wolf[key] = X2
            elif key in delta:
                D_delta = abs(C * delta[key] - wolf[key])
                X3 = delta[key] - A * D_delta
                wolf[key] = X3
            else:
                choice = random.choice([alpha, beta, delta])
                D_choice = abs(C * choice[key] - wolf[key])
                X_choice = choice[key] - A * D_choice
                wolf[key] = X_choice

def evaluate_fitness(population):
    fitness_scores = []
    for wolf in population:
        fitness_scores.append(objective_function(wolf))
    return fitness_scores

# Termination criteria
max_iterations = 50 
convergence_threshold = 1e-6 # to 1e-4
max_consecutive_iterations = 5

# Initialize best_params
best_hyperparameters = {
    'max_depth': 5,
    'learning_rate': 0.5,
    'n_estimators': 100,
    'min_child_weight': 1.0,
    'scale_pos_weight':1,
    'gamma': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'max_leaf_nodes': 5,
    'seed': 42
}

best_fitness = objective_function(best_hyperparameters)
consecutive_iterations = 5 #3,7,10

# Main GWO optimization 
for iteration in range(max_iterations):
    # Initialize alpha, beta, delta wolves
    alpha, beta, delta = None, None, None
    population = initialize_population(population_size=50)  
    
    a = 2.0
    A = 2 * a * (1 - (iteration / max_iterations))  
    
    fitness_scores = evaluate_fitness(population)
    alpha_idx = fitness_scores.index(min(fitness_scores))
    alpha = population[alpha_idx].copy()
    beta_idx = fitness_scores.index(sorted(fitness_scores)[1])
    beta = population[beta_idx].copy()
    delta_idx = fitness_scores.index(sorted(fitness_scores)[2])
    delta = population[delta_idx].copy()
    
    update_wolf_positions(population, alpha, beta, delta, a, A, 2 * random.random() - 1)
    
    best_hyperparameters = alpha  
    
    current_fitness = objective_function(best_hyperparameters)
    
    if current_fitness > best_fitness:
        best_fitness = current_fitness
        consecutive_iterations = 0
    else:
        consecutive_iterations += 1
    
    if consecutive_iterations >= max_consecutive_iterations:
        print("Convergence criteria met. Terminating optimization.")
        break
    
    if iteration == max_iterations - 1:
        print("Maximum iterations reached. Terminating optimization.")

print("Final Hyperparameters:", best_hyperparameters)

In [ ]:
xgb_classifier = xgb.XGBClassifier(n_estimators=150, learning_rate = 0.04475, max_depth = 8, min_child_weight= 0.8954022553993488, subsample= 0.7513841858486106, 
                        max_leaf_nodes = 5, gamma = 0.1, seed = 42)

In [ ]:
%%time
xgb_classifier.fit(encoded_features_train_ENC, y_train_target) 

In [ ]:
%%time
y_predxgp = xgb_classifier.predict(encoded_features_test_ENC) 

In [ ]:
# Define the confusion matrix heatmap function
def confusionMatrixHeatMap(cm, title):
    
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_percentage = cm / cm_sum * 100

    labels = np.array([["{}\n{:.3f}%".format(value, percentage) for value, percentage in zip(row, percentage_row)]
                       for row, percentage_row in zip(cm, cm_percentage)])

    # Categories
    categories = ['Benign', 'Malicious']

    max_value = np.max(cm)  

    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    sns.heatmap(cm, annot=labels, fmt='', cmap='Blues', cbar=True,
                annot_kws={"size": 14}, linewidths=2, linecolor='black',
                xticklabels=categories, yticklabels=categories, vmin=0, vmax=max_value)
    # Set labels, title, and axis ticks
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(title)

    #plt.savefig('confusion_matrix_GWO_xgboost.png')
    plt.show()

print("XGBoost")

# Generate report
cm = confusion_matrix(y_test_target, y_predxgp )
cr = classification_report(y_test_target, y_predxgp )
auc = roc_auc_score(y_test_target, y_predxgp )

# calculate accuracy and misclassification rate
total_samples = len(y_test_target)
correct_predictions = sum(y_test_target == y_predxgp)
incorrect_predictions = total_samples - correct_predictions
accuracy = correct_predictions / total_samples
misclassification_rate = incorrect_predictions / total_samples

plt.text(0.5, -0.1, f'\n\n\nAccuracy: {accuracy:.4f}', ha='center', va='center', transform=plt.gca().transAxes)
plt.text(0.5, -0.2, f'\nMisclassification Rate: {misclassification_rate:.4f}', ha='center', va='center', transform=plt.gca().transAxes)

print("Confusion Matrix:")
print(cm)

print("Performance Matrix:")
print(cr)

print("AUC:", auc)
confusionMatrixHeatMap(cm, title="GWO-Optimized XGBoost")
#plt.savefig('CICIDS2017_CM.png')

In [ ]:
# Calculate ROC curve
fpr, tpr, _ = roc_curve(y_test_target, y_predxgp)
roc_auc = auc(fpr, tpr)

# plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')

plt.text(0.5, -0.1, f'\n\n\nAccuracy: {accuracy:.4f}', ha='center', va='center', transform=plt.gca().transAxes)
plt.text(0.5, -0.2, f'\nMisclassification Rate: {misclassification_rate:.4f}', ha='center', va='center', transform=plt.gca().transAxes)
#plt.savefig('ROC_OPXGB_.png')
plt.show()

In [ ]:
#Training and Validation Accuracy
xgb_classifier = xgb.XGBClassifier(n_estimators=150, learning_rate = 0.04475, max_depth = 8, min_child_weight= 0.8954022553993488, subsample= 0.7513841858486106, 
                        max_leaf_nodes = 5, gamma = 0.1, seed = 42)

# Fit the model with evaluation sets
evals = [(encoded_features_train_ENC, encoded_y_train), (encoded_features_test_ENC, encoded_y_test)]
xgb_classifier.fit(encoded_features_train_ENC, encoded_y_train, eval_metric=["error", "logloss"], eval_set=evals, verbose=True)

results = xgb_classifier.evals_result()

train_accuracy = 1 - np.array(results["validation_0"]["error"])
val_accuracy = 1 - np.array(results["validation_1"]["error"])

epochs = range(1, len(train_accuracy) + 1)
plt.plot(epochs, train_accuracy, label="Training Accuracy")
plt.plot(epochs, val_accuracy, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy over Epochs")
plt.ylim([0, 1.0])
plt.legend()
plt.show()

Xgbb_pred = xgb_classifier.predict(encoded_features_test_ENC)
accuracy = accuracy_score(encoded_y_test, Xgbb_pred)
print("Test Accuracy:", accuracy)
cr = classification_report(encoded_y_test, Xgbb_pred)
print(cr)